In [15]:
import pandas as pd
import numpy as np

In [16]:
# 1. LOAD RAW DATA
df = pd.read_csv("azure_demand_raw.csv")

print("Initial shape:", df.shape)
print(df.head())

Initial shape: (5700, 10)
             timestamp        region service_type  usage_units  \
0  2023-03-12 01:14:50       us-west      Storage      8824.65   
1  2023-01-31 10:18:59  Europe-North      Compute     13098.88   
2  2023-08-30 21:38:21       us-east      Storage          NaN   
3  2024-02-19 05:23:30       us east      Storage      6408.14   
4  2024-03-19 21:40:00       us-west      Compute     11940.33   

   provisioned_capacity  cost_usd  availability_pct  cloud_spend_index  \
0                  8000   2823.89             99.94              92.91   
1                 10000   6287.46             99.69             103.90   
2                  8000       NaN             99.57              90.63   
3                  8000   2050.61             99.46             100.01   
4                 10000   5731.36             99.89              96.41   

   enterprise_demand_index  new_service_launch  
0                     1.21                   0  
1                     1.35        

In [6]:
# 2. FIX TIMESTAMP
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df = df.dropna(subset=['timestamp'])
df = df.sort_values('timestamp')

In [8]:
# 3. REMOVE DUPLICATES
df = df.drop_duplicates()
print("After removing duplicates:", df.shape)

After removing duplicates: (5500, 10)


In [9]:
# 4. STANDARDIZE REGION NAMES
df['region'] = (
    df['region']
    .str.strip()
    .str.lower()
    .str.replace(" ", "-", regex=False)
)

df['region'] = df['region'].replace({
    'us-east': 'US-East',
    'us-west': 'US-West',
    'india-south': 'India-South',
    'europe-north': 'Europe-North'
})


In [10]:
# 5. STANDARDIZE SERVICE TYPE
df['service_type'] = df['service_type'].str.title()

In [11]:
# 6. HANDLE MISSING VALUES

# Time-series interpolation for usage
df['usage_units'] = df['usage_units'].interpolate(method='linear')

# Cost recompute if missing
df['cost_usd'] = df['cost_usd'].fillna(df['usage_units'] * 0.45)

# Availability forward fill
df['availability_pct'] = df['availability_pct'].ffill()

# Economic variables --> median
df['cloud_spend_index'] = df['cloud_spend_index'].fillna(
    df['cloud_spend_index'].median()
)

df['enterprise_demand_index'] = df['enterprise_demand_index'].fillna(
    df['enterprise_demand_index'].median()
)

# Technical binary variable
df['new_service_launch'] = df['new_service_launch'].fillna(0)


In [12]:
# 7. VALIDATION RULES

# Remove unrealistic usage
df = df[df['usage_units'] > 0]

# Availability must be realistic
df = df[(df['availability_pct'] >= 95) & (df['availability_pct'] <= 100)]

# Remove negative costs
df = df[df['cost_usd'] > 0]

print("\nRemaining NaNs:\n", df.isnull().sum())
print("Final shape after cleaning:", df.shape)



Remaining NaNs:
 timestamp                  0
region                     0
service_type               0
usage_units                0
provisioned_capacity       0
cost_usd                   0
availability_pct           0
cloud_spend_index          0
enterprise_demand_index    0
new_service_launch         0
dtype: int64
Final shape after cleaning: (5500, 10)


In [13]:
# 8. SAVE CLEAN DATA
df.to_csv("azure_demand_cleaned.csv", index=False)

print("Clean dataset saved successfully.")


Clean dataset saved successfully.
